# Exploration of how to access this d*mn data

In [40]:
import os
from os.path import join
import argparse
import pandas as pd
import re
import numpy as np
from datetime import datetime


## get going

In [2]:
os.getcwd()

'/Users/thearolskovsloth/gutenberg.nosync/data-science-x-am/src'

In [4]:
metadata = pd.read_csv(os.path.join("..","..","metadata", "metadata.csv"))
meta = pd.DataFrame(metadata)

In [5]:
type([meta['language']])
meta.head(5)



,id,title,author,authoryearofbirth,authoryearofdeath,language,downloads,subjects,type
0,PG10000,The Magna Carta,Anonymous,NaN,NaN,['en'],171,{'Constitutional history -- England -- Sources...,NaN
1,PG10001,Apocolocyntosis,"Seneca, Lucius Annaeus",NaN,65.0,['en'],262,"{'Claudius, Emperor of Rome, 10 B.C.-54 A.D. -...",NaN
2,PG10002,The House on the Borderland,"Hodgson, William Hope",1877.0,1918.0,['en'],488,{'Science fiction'},NaN
3,PG10003,"My First Years as a Frenchwoman, 1876-1879","Waddington, Mary King",1833.0,1923.0,['en'],8,"{'France -- Social life and customs', 'France ...",NaN
4,PG10004,The Warriors,"Lindsay, Anna Robertson Brown",1864.0,1948.0,['en'],12,{'Christianity'},NaN


In [7]:
different_languages = meta['language'].value_counts()
print(different_languages)
print("number of diff languages:", len(different_languages))


['en']                54701
['fr']                 3408
['fi']                 2474
['de']                 2020
['it']                  909
                      ...  
['mi']                    1
['de', 'ca']              1
['en', 'myn']             1
['en', 'tl', 'es']        1
['en', 'ang']             1
Name: language, Length: 130, dtype: int64
number of diff languages: 130


In [8]:
#selecting only english texts
eng_meta = meta.loc[meta['language'] == "['en']"]
print(eng_meta['language'].value_counts())
eng_meta.head(5)
eng_meta.tail(5)


['en']    54701
Name: language, dtype: int64


,id,title,author,authoryearofbirth,authoryearofdeath,language,downloads,subjects,type
67838,PG9997,"France and England in North America, Part III:...","Parkman, Francis",1823.0,1893.0,['en'],17,{'Canada -- History -- To 1763 (New France)'},NaN
67839,PG9998,Poems,"Betham, Matilda",1776.0,1852.0,['en'],70,{'Poetry'},NaN
67840,PG9999,"Harriet, the Moses of Her People","Bradford, Sarah H. (Sarah Hopkins)",1818.0,NaN,['en'],85,"{'Slaves -- United States -- Biography', 'Afri...",NaN
67842,PG99,Collected Articles of Frederick Douglass,"Douglass, Frederick",1818.0,1895.0,['en'],178,"{'African American abolitionists', 'Essays', '...",NaN
67843,PG9,Abraham Lincoln's First Inaugural Address,"Lincoln, Abraham",1809.0,1865.0,['en'],157,{'United States -- Politics and government -- ...,NaN


In [9]:
handles = eng_meta.id.unique()
print(len(handles))

54701


In [10]:

def get_brodtext_from_handle(handle,path):

    """
    Fetches the text from a handle
    
    """
    with open(f"{path}{handle}_text.txt") as f:
        brodtext = f.read()
    return brodtext

test_path = "../../gutenberg.nosync/data/text/"
test_handles = ['PG100', 'PG102']

for book in test_handles:
    f = get_brodtext_from_handle(book, test_path)

print(f)



FileNotFoundError: [Errno 2] No such file or directory: '../../gutenberg.nosync/data/text/PG100_text.txt'

In [11]:
ano = eng_meta[eng_meta.author == 'Anonymous']
len(ano)

#uh. der er 705 anonyme - dem fjerner vi
eng_meta = eng_meta[eng_meta.author != 'Anonymous']

ano = eng_meta[eng_meta.author == 'Anonymous']
len(ano) #REMOVED

0

In [12]:
various = eng_meta[eng_meta.author == 'Various']
len(various)

#uh. der er 3417 various - dem fjerner vi
eng_meta = eng_meta[eng_meta.author != 'Various']

In [13]:
eng_meta = eng_meta.dropna(subset=['author'])
eng_meta.head(5)

,id,title,author,authoryearofbirth,authoryearofdeath,language,downloads,subjects,type
1,PG10001,Apocolocyntosis,"Seneca, Lucius Annaeus",NaN,65.0,['en'],262,"{'Claudius, Emperor of Rome, 10 B.C.-54 A.D. -...",NaN
2,PG10002,The House on the Borderland,"Hodgson, William Hope",1877.0,1918.0,['en'],488,{'Science fiction'},NaN
3,PG10003,"My First Years as a Frenchwoman, 1876-1879","Waddington, Mary King",1833.0,1923.0,['en'],8,"{'France -- Social life and customs', 'France ...",NaN
4,PG10004,The Warriors,"Lindsay, Anna Robertson Brown",1864.0,1948.0,['en'],12,{'Christianity'},NaN
5,PG10005,A Voyage to the Moon: With Some Account of the...,"Tucker, George",1775.0,1861.0,['en'],30,"{'Space flight to the moon -- Fiction', 'Scien...",NaN


In [14]:
#fetching names to determine gender using regex
findmary= eng_meta.loc[4,'author']
mary =re.findall(r'(?<=,\s)[a-z]+',findmary,re.I)
MARY = mary[0]
type(MARY)

str

In [20]:
#create empty column for extracting the name
eng_meta["name"] = None

#get the first names after comma
eng_meta['name'] = eng_meta.apply(lambda row: re.findall(r'(?<=,\s)[a-z]+',row.author,re.I), axis=1)

#get the first element of the first name and asribe NaN to the emtpy lists
eng_meta['name'] = [l[0] if len(l) > 0 else np.nan for l in eng_meta['name']]

#remove the nans
eng_meta = eng_meta.dropna(subset=['name'])

#if there is only one letter, put NaN
eng_meta['name'] = [l if len(l) > 1 else np.nan for l in eng_meta['name']]

eng_meta['name'] = eng_meta['name'].str.lower()

In [16]:
#remove
eng_meta = eng_meta.dropna(subset=['authoryearofbirth'])

In [21]:

eng_meta

,id,title,author,authoryearofbirth,authoryearofdeath,language,downloads,subjects,type,name
2,PG10002,The House on the Borderland,"Hodgson, William Hope",1877.0,1918.0,['en'],488,{'Science fiction'},NaN,william
3,PG10003,"My First Years as a Frenchwoman, 1876-1879","Waddington, Mary King",1833.0,1923.0,['en'],8,"{'France -- Social life and customs', 'France ...",NaN,mary
4,PG10004,The Warriors,"Lindsay, Anna Robertson Brown",1864.0,1948.0,['en'],12,{'Christianity'},NaN,anna
5,PG10005,A Voyage to the Moon: With Some Account of the...,"Tucker, George",1775.0,1861.0,['en'],30,"{'Space flight to the moon -- Fiction', 'Scien...",NaN,george
6,PG10006,La Fiammetta,"Boccaccio, Giovanni",1313.0,1375.0,['en'],59,{'Fiction'},NaN,giovanni
...,...,...,...,...,...,...,...,...,...,...
67838,PG9997,"France and England in North America, Part III:...","Parkman, Francis",1823.0,1893.0,['en'],17,{'Canada -- History -- To 1763 (New France)'},NaN,francis
67839,PG9998,Poems,"Betham, Matilda",1776.0,1852.0,['en'],70,{'Poetry'},NaN,matilda
67840,PG9999,"Harriet, the Moses of Her People","Bradford, Sarah H. (Sarah Hopkins)",1818.0,NaN,['en'],85,"{'Slaves -- United States -- Biography', 'Afri...",NaN,sarah
67842,PG99,Collected Articles of Frederick Douglass,"Douglass, Frederick",1818.0,1895.0,['en'],178,"{'African American abolitionists', 'Essays', '...",NaN,frederick


# Assign gender

In [23]:
gender = pd.read_csv(os.path.join("..","..","metadata","name_gender.csv"))
gender["name"] =gender["name"].str.lower()
gender

,name,gender,probability
0,aaban,M,1.0
1,aabha,F,1.0
2,aabid,M,1.0
3,aabriella,F,1.0
4,aada,F,1.0
...,...,...,...
95021,zyvon,M,1.0
95022,zyyanna,F,1.0
95023,zyyon,M,1.0
95024,zzyzx,M,1.0


In [24]:
#create a new variable "name" which is the first part of every name (splitting by -)
data = pd.merge(eng_meta, gender, on=['name'], how = 'left')            
# merge two datasets by the name variable so that every adam gets a male, etc
data

,id,title,author,authoryearofbirth,authoryearofdeath,language,downloads,subjects,type,name,gender,probability
0,PG10002,The House on the Borderland,"Hodgson, William Hope",1877.0,1918.0,['en'],488,{'Science fiction'},NaN,william,M,0.996107
1,PG10003,"My First Years as a Frenchwoman, 1876-1879","Waddington, Mary King",1833.0,1923.0,['en'],8,"{'France -- Social life and customs', 'France ...",NaN,mary,F,0.996333
2,PG10004,The Warriors,"Lindsay, Anna Robertson Brown",1864.0,1948.0,['en'],12,{'Christianity'},NaN,anna,F,0.996908
3,PG10005,A Voyage to the Moon: With Some Account of the...,"Tucker, George",1775.0,1861.0,['en'],30,"{'Space flight to the moon -- Fiction', 'Scien...",NaN,george,M,0.993236
4,PG10006,La Fiammetta,"Boccaccio, Giovanni",1313.0,1375.0,['en'],59,{'Fiction'},NaN,giovanni,M,0.985996
...,...,...,...,...,...,...,...,...,...,...,...,...
40112,PG9997,"France and England in North America, Part III:...","Parkman, Francis",1823.0,1893.0,['en'],17,{'Canada -- History -- To 1763 (New France)'},NaN,francis,M,0.907432
40113,PG9998,Poems,"Betham, Matilda",1776.0,1852.0,['en'],70,{'Poetry'},NaN,matilda,F,0.999670
40114,PG9999,"Harriet, the Moses of Her People","Bradford, Sarah H. (Sarah Hopkins)",1818.0,NaN,['en'],85,"{'Slaves -- United States -- Biography', 'Afri...",NaN,sarah,F,0.996901
40115,PG99,Collected Articles of Frederick Douglass,"Douglass, Frederick",1818.0,1895.0,['en'],178,"{'African American abolitionists', 'Essays', '...",NaN,frederick,M,0.995759


In [25]:
data = data.drop(columns=['type', 'probability'])
data

,id,title,author,authoryearofbirth,authoryearofdeath,language,downloads,subjects,name,gender
0,PG10002,The House on the Borderland,"Hodgson, William Hope",1877.0,1918.0,['en'],488,{'Science fiction'},william,M
1,PG10003,"My First Years as a Frenchwoman, 1876-1879","Waddington, Mary King",1833.0,1923.0,['en'],8,"{'France -- Social life and customs', 'France ...",mary,F
2,PG10004,The Warriors,"Lindsay, Anna Robertson Brown",1864.0,1948.0,['en'],12,{'Christianity'},anna,F
3,PG10005,A Voyage to the Moon: With Some Account of the...,"Tucker, George",1775.0,1861.0,['en'],30,"{'Space flight to the moon -- Fiction', 'Scien...",george,M
4,PG10006,La Fiammetta,"Boccaccio, Giovanni",1313.0,1375.0,['en'],59,{'Fiction'},giovanni,M
...,...,...,...,...,...,...,...,...,...,...
40112,PG9997,"France and England in North America, Part III:...","Parkman, Francis",1823.0,1893.0,['en'],17,{'Canada -- History -- To 1763 (New France)'},francis,M
40113,PG9998,Poems,"Betham, Matilda",1776.0,1852.0,['en'],70,{'Poetry'},matilda,F
40114,PG9999,"Harriet, the Moses of Her People","Bradford, Sarah H. (Sarah Hopkins)",1818.0,NaN,['en'],85,"{'Slaves -- United States -- Biography', 'Afri...",sarah,F
40115,PG99,Collected Articles of Frederick Douglass,"Douglass, Frederick",1818.0,1895.0,['en'],178,"{'African American abolitionists', 'Essays', '...",frederick,M


## SAVE

In [37]:
males = data[data.gender == "M" ]
len(males)

27271

In [47]:
#remove rows where a gender was not assigned
data = data.drop(data[(data['gender'] != 'M') & (data['gender'] != 'F')].index)

In [48]:
# get time point right now
now = datetime.now()
dt_string = str(now.strftime("%d-%m-%Y%H:%M:%S"))

#write preprocessed file with time stamp in file name
data.to_csv(os.path.join("..","..","metadata",f"meta_gender_en_{dt_string}.csv")) 
print("preprocessing finished") 

preprocessing finished
